In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install seaborn
import seaborn as sns

from summit.benchmarks import  (
        MIT_case1, MIT_case2, MIT_case3,
        MIT_case4, MIT_case5,
)
from summit.strategies import LHS

You should consider upgrading via the '/home/riley/Software/anaconda3/envs/torch/bin/python -m pip install --upgrade pip' command.


## Kinetic model

For a general bi-molecular reaction $A+B \rightarrow R$ catalyzed by a transition metal complex with concentration $C_{\text{cat}}$, the objective function can be defined as follows assuming a closed system with a constant reaction volume. The product yield is calculated based on the initial concentration of $A$, $C_{A_o}$,

$$ \phi (\mathbf{x}, \mathbf{y}) = \log(\text{TON}) = \log \left( \frac{C_R}{C_{\text{cat}}} \right)$$

$$ Y (\mathbf{x}, \mathbf{y}) = \frac{C_R}{C_{A_0}}  $$

In the original paper (DOI: 10.1039/c8re00032h), the authors select the economic use of catalyst, i.e. maximization of the TON as the primary objective, subject to the yield being greater than some threshold value. 

The full form of the equations are as follows

$$ \log(\text{TON}) = \log \left( \frac{C_R}{C_{\text{cat}}} \right) \alpha \log(A_i) - \frac{E_{A_i}}{R} \frac{1}{T} - \frac{E_{A_R}}{R} \frac{1}{T} + (r-1) \log(C_{\text{cat}}) + \log(t_{\text{res}})
$$

$$ \log(Y) = \log \left( \frac{C_R}{C_{A_0}} \right) \alpha \log(A_i) - \frac{E_{A_i}}{R} \frac{1}{T} - \frac{E_{A_R}}{R} \frac{1}{T} + r \log(C_{\text{cat}}) + \log(t_{\text{res}})
$$


The authors study five specififc cases 


|  Case   | Catalyst effect  |    $k_{S_1}$  |  $k_{S_2}$ |
|:--------|:-----------------|:--------------|:-----------|
|One optimum: 1 | $E_{A_1} > E_{A_{2-8}}$ | $=0$ | $=0$ | 



The optimization consists of 1 categorical/discrete parameter and 3 continuous process parameters

* Catalyst ('1'-'8')
* Temperature, T (30 - 110 degC)
* Reaction time, $t_{\text{res}}$ (1 - 10 min)
* Catalyst concentration, $C_{\text{cat}}$ (0.835-4.175mM or  0.5-2.5 mol%)


In [3]:
# helper function to convert summit params to mnemosyne 

def convert_params(sub_df):
    conc_cat = sub_df['conc_cat'].values.reshape(-1,1)
    time = sub_df['t'].values.reshape(-1, 1)
    cat_index = sub_df['cat_index'].values
    temp = sub_df['temperature'].values.reshape(-1, 1)
    
    # convert cat indices to vectors
    hots = []
    for cat_ix in cat_index:
        cat_ix = int(cat_ix)
        vec = np.zeros(8)
        vec[cat_ix]+=1.
        hots.append(vec)
    hots = np.array(hots)

    return np.concatenate((hots, temp, time, conc_cat), axis=1)


def select_at_least_one_cat(pt_data, num_points):
    ''' make sure selection for source task has at least one of
    all 8 types of ligand/catalyst. If not, this will spell
    problems for the normalization of the parameters within the
    meta planners.
    '''
    is_sat = False
    while not is_sat:
        # select random subset of pt_data
        indices = np.arange(pt_data.shape[0])
        np.random.shuffle(indices)
        ind = indices[:num_points]
        sub_df = pt_data.iloc[ind, :]
        if len(list(set(sub_df['cat_index']))) == 8:
            is_sat = True
    return sub_df
    

In [4]:
# generate test data for meta-learning planners for each case
NUM_POINTS = 300
NUM_RETURN = 8   # number of points per sample per source task
NUM_SAMPLES = 40 # number of source task samples


all_task_samples = []

for sample_ix in range(NUM_SAMPLES):

    # generate new random state
    RANDOM_STATE = np.random.randint(0, 10e6) #100700
    print(f'SAMPLE IX : {sample_ix}\tRANDOM STATE : {RANDOM_STATE}')
    #-------
    # case 1
    #-------

    exp_pt = MIT_case1(noise_level=1)
    random_state = np.random.RandomState(RANDOM_STATE)
    planner = LHS(exp_pt.domain, random_state=random_state)

    conditions = planner.suggest_experiments(NUM_POINTS)

    exp_pt.run_experiments(conditions)
    pt_data_1 = exp_pt.data

    pt_data_1 = select_at_least_one_cat(pt_data_1, NUM_RETURN)

    params = convert_params(pt_data_1) # shape (# observations, 11)
    values = pt_data_1['y'].values.reshape(-1, 1)

    case1_task = {'params': params, 'values': values}

#    print(pt_data_1.shape, params.shape, values.shape)

    #-------
    # case 2
    #-------

    exp_pt = MIT_case2(noise_level=1)
    random_state = np.random.RandomState(RANDOM_STATE)
    planner = LHS(exp_pt.domain, random_state=random_state)

    conditions = planner.suggest_experiments(NUM_POINTS)

    exp_pt.run_experiments(conditions)
    pt_data_2 = exp_pt.data

    pt_data_2 = select_at_least_one_cat(pt_data_2, NUM_RETURN)

    params = convert_params(pt_data_2) # shape (# observations, 11)
    values = pt_data_2['y'].values.reshape(-1, 1)

    case2_task = {'params': params, 'values': values}

#    print(pt_data_2.shape, params.shape, values.shape)


    #-------
    # case 3
    #-------

    exp_pt = MIT_case3(noise_level=1)
    random_state = np.random.RandomState(RANDOM_STATE)
    planner = LHS(exp_pt.domain, random_state=random_state)

    conditions = planner.suggest_experiments(NUM_POINTS)

    exp_pt.run_experiments(conditions)
    pt_data_3 = exp_pt.data

    pt_data_3 = select_at_least_one_cat(pt_data_3, NUM_RETURN)

    params = convert_params(pt_data_3) # shape (# observations, 11)
    values = pt_data_3['y'].values.reshape(-1, 1)

    case3_task = {'params': params, 'values': values}

#    print(pt_data_2.shape, params.shape, values.shape)


    #-------
    # case 4
    #-------

    exp_pt = MIT_case4(noise_level=1)
    random_state = np.random.RandomState(RANDOM_STATE)
    planner = LHS(exp_pt.domain, random_state=random_state)

    conditions = planner.suggest_experiments(NUM_POINTS)

    exp_pt.run_experiments(conditions)
    pt_data_4 = exp_pt.data

    pt_data_4 = select_at_least_one_cat(pt_data_4, NUM_RETURN)

    params = convert_params(pt_data_4) # shape (# observations, 11)
    values = pt_data_4['y'].values.reshape(-1, 1)

    case4_task = {'params': params, 'values': values}

#    print(pt_data_4.shape, params.shape, values.shape)


    #-------
    # case 5
    #-------

    exp_pt = MIT_case5(noise_level=1)
    random_state = np.random.RandomState(RANDOM_STATE)
    planner = LHS(exp_pt.domain, random_state=random_state)

    conditions = planner.suggest_experiments(NUM_POINTS)

    exp_pt.run_experiments(conditions)
    pt_data_5 = exp_pt.data

    pt_data_5 = select_at_least_one_cat(pt_data_5, NUM_RETURN)

    params = convert_params(pt_data_5) # shape (# observations, 11)
    values = pt_data_5['y'].values.reshape(-1, 1)

    case5_task = {'params': params, 'values': values}

#    print(pt_data_5.shape, params.shape, values.shape)

    # add sample
    all_task_samples.append([case1_task, case2_task, case3_task, case4_task, case5_task])

SAMPLE IX : 0	RANDOM STATE : 448050
SAMPLE IX : 1	RANDOM STATE : 9767434
SAMPLE IX : 2	RANDOM STATE : 5875118
SAMPLE IX : 3	RANDOM STATE : 8411757
SAMPLE IX : 4	RANDOM STATE : 7962162
SAMPLE IX : 5	RANDOM STATE : 9430438
SAMPLE IX : 6	RANDOM STATE : 9223057
SAMPLE IX : 7	RANDOM STATE : 7335278
SAMPLE IX : 8	RANDOM STATE : 5983079
SAMPLE IX : 9	RANDOM STATE : 8593508
SAMPLE IX : 10	RANDOM STATE : 4373633
SAMPLE IX : 11	RANDOM STATE : 8100441
SAMPLE IX : 12	RANDOM STATE : 6541466
SAMPLE IX : 13	RANDOM STATE : 1027501
SAMPLE IX : 14	RANDOM STATE : 1194260
SAMPLE IX : 15	RANDOM STATE : 149048
SAMPLE IX : 16	RANDOM STATE : 6868645
SAMPLE IX : 17	RANDOM STATE : 4371917
SAMPLE IX : 18	RANDOM STATE : 1905597
SAMPLE IX : 19	RANDOM STATE : 4294464
SAMPLE IX : 20	RANDOM STATE : 4352916
SAMPLE IX : 21	RANDOM STATE : 7836205
SAMPLE IX : 22	RANDOM STATE : 6142514
SAMPLE IX : 23	RANDOM STATE : 879331
SAMPLE IX : 24	RANDOM STATE : 4179944
SAMPLE IX : 25	RANDOM STATE : 4258427
SAMPLE IX : 26	RANDOM STA

In [6]:
# save all of the tasks to the disk
pickle.dump(all_task_samples, open('tasks_8.pkl', 'wb'))

## Analysis of the correlation between tasks

In [ ]:
from scipy.stats import pearsonr, spearmanr

In [ ]:
corr12 = pearsonr(
    case1_task['values'].ravel(), case2_task['values'].ravel()
)[0]
corr12

In [ ]:
corr13 = pearsonr(
    case1_task['values'].ravel(), case3_task['values'].ravel()
)[0]
corr13

In [ ]:
corr14 = pearsonr(
    case1_task['values'].ravel(), case4_task['values'].ravel()
)[0]
corr14

In [ ]:
corr15 = pearsonr(
    case1_task['values'].ravel(), case5_task['values'].ravel()
)[0]
corr15

## Analysis of the locations of the optimia between tasks

In [ ]:
argmax_1 = np.argmax(case1_task['values'])
max_yield_1 = case1_task['values'][argmax_1]
opt_params_1 = case1_task['params'][argmax_1, :]

opt_params_1, max_yield_1

In [ ]:
argmax_2 = np.argmax(case2_task['values'])
max_yield_2 = case2_task['values'][argmax_2]
opt_params_2 = case2_task['params'][argmax_2, :]

opt_params_2, max_yield_2

In [ ]:
argmax_3 = np.argmax(case3_task['values'])
max_yield_3 = case3_task['values'][argmax_3]
opt_params_3 = case3_task['params'][argmax_3, :]

opt_params_3, max_yield_3

In [ ]:
argmax_4 = np.argmax(case4_task['values'])
max_yield_4 = case4_task['values'][argmax_4]
opt_params_4 = case4_task['params'][argmax_4, :]

opt_params_4, max_yield_4

In [ ]:
argmax_5 = np.argmax(case5_task['values'])
max_yield_5 = case5_task['values'][argmax_5]
opt_params_5 = case5_task['params'][argmax_5, :]

opt_params_5, max_yield_5

In [ ]:
pt_data_1.sort_values(by='y', ascending=False).iloc[:15, :]

In [ ]:
pt_data_2.sort_values(by='y', ascending=False).head()

In [ ]:
pt_data_3.sort_values(by='y', ascending=False).head()

In [ ]:
pt_data_4.sort_values(by='y', ascending=False).head()

In [ ]:
pt_data_5.sort_values(by='y', ascending=False).head()

In [ ]:
# rearrage the columns to be in same order as optimization
cols = ['cat_index', 'temperature', 't', 'conc_cat', 'y', 'computation_t', 'experiment_t', 'strategy']

pt_data_1 = pt_data_1[cols]
pt_data_2 = pt_data_2[cols]
pt_data_3 = pt_data_3[cols]
pt_data_4 = pt_data_4[cols]
pt_data_5 = pt_data_5[cols]


In [ ]:
pt_data_1_hot = convert_params(pt_data_1)
pt_data_1_hot.shape

In [ ]:
train_params_1 = convert_params(pt_data_1) #pt_data_1.iloc[:, :4].values
train_values_1 = pt_data_1.iloc[:, 4].values.reshape(-1, 1)

train_params_2 = convert_params(pt_data_2) #pt_data_2.iloc[:, :4].values
train_values_2 = pt_data_2.iloc[:, 4].values.reshape(-1, 1)

train_params_3 = convert_params(pt_data_3) #pt_data_3.iloc[:, :4].values
train_values_3 = pt_data_3.iloc[:, 4].values.reshape(-1, 1)

train_params_4 = convert_params(pt_data_4) #pt_data_4.iloc[:, :4].values
train_values_4 = pt_data_4.iloc[:, 4].values.reshape(-1, 1)

train_params_5 = convert_params(pt_data_5) #pt_data_5.iloc[:, :4].values
train_values_5 = pt_data_5.iloc[:, 4].values.reshape(-1, 1)


tasks = [
    {'params': train_params_1, 'values': train_values_1},
    {'params': train_params_2, 'values': train_values_2},
    {'params': train_params_3, 'values': train_values_3},
    {'params': train_params_4, 'values': train_values_4},
    {'params': train_params_5, 'values': train_values_5},
    
]
pickle.dump(tasks, open('tasks_10.pkl', 'wb'))

In [ ]:
from neural_processes.nps import NeuralProcess
from neural_processes.observation_processor import Normalizer

In [ ]:
normalizer = Normalizer()


In [ ]:
train_params_1 = normalizer.scalarize(train_params_1)
train_values_1 = normalizer.scalarize(train_values_1)

train_params_2 = normalizer.scalarize(train_params_2)
train_values_2 = normalizer.scalarize(train_values_2)

train_params_3 = normalizer.scalarize(train_params_3)
train_values_3 = normalizer.scalarize(train_values_3)

train_params_4 = normalizer.scalarize(train_params_4)
train_values_4 = normalizer.scalarize(train_values_4)

train_params_5 = normalizer.scalarize(train_params_5)
train_values_5 = normalizer.scalarize(train_values_5)

In [ ]:
tasks = [
    {'params': train_params_1, 'values': train_values_1},
    {'params': train_params_2, 'values': train_values_2},
    {'params': train_params_3, 'values': train_values_3},
    {'params': train_params_4, 'values': train_values_4},
    {'params': train_params_5, 'values': train_values_5},
    
]

tasks[0]['params']

In [ ]:
hyperparams = {
    'model': {'learning_rate': 8e-4, 'epochs': 10000}
}

model = NeuralProcess(
            x_dim=4,
            y_dim=1, 
            use_self_attention=False, 
            use_cross_attention=True,
            hyperparams=hyperparams,
)

model.train(tasks[1:], tasks[1:])

## Run the multi-task BayesOpt experiments with Summit

In [ ]:
from summit.strategies import MTBO, STBO, Transform, LHS, Chimera
from summit.utils.dataset import DataSet
from summit.domain import *
import summit

from IPython.display import clear_output
from typing import List

In [ ]:
def run_stbo(exp, max_iterations=10, categorical_method="one-hot"):
    exp.reset()
    strategy = STBO(exp.domain, 
                    categorical_method=categorical_method)
    r = summit.Runner(strategy=strategy, 
                      experiment=exp, 
                      max_iterations=max_iterations)
    r.run()
    return r

def run_mtbo(exp, pt_data, max_iterations=10):
    strategy = MTBO(exp.domain, 
                    pretraining_data=pt_data,
                    categorical_method="one-hot", 
                    task=1)
    r = summit.Runner(strategy=strategy,
                      experiment=exp, 
                      max_iterations=max_iterations)
    r.run()
    return r

def make_average_plot(results: List[summit.Runner], ax, label=None, color=None):
    objective = results[0].experiment.domain.output_variables[0].name
    yields = [r.experiment.data[objective] for r in results]
    yields = np.array(yields)
    mean_yield = np.mean(yields, axis=0)
    std_yield = np.std(yields, axis=0)
    x = np.arange(0, len(mean_yield), 1).astype(int)
    ax.plot(x, mean_yield, label=label, linewidth=2)
    ax.fill_between(x, mean_yield-std_yield, mean_yield+std_yield, alpha=0.1)
    



def make_comparison_plot(*args):
    fig, ax = plt.subplots(1)
    for arg in args:
        make_average_plot(arg['results'], ax, label=arg["label"], color=arg.get("color"))
    fontdict = fontdict={"size":12}
    ax.legend(loc = "lower right", prop=fontdict)
    ax.set_xlim(0,20)
    ax.set_xticks(np.arange(0, 20, 2).astype(int))
    ax.set_ylabel('Yield', fontdict=fontdict)
    ax.set_xlabel('Reactions', fontdict=fontdict)
    ax.tick_params(direction='in')
    return fig, ax

In [ ]:
NUM_REPEATS = 10
MAX_ITER = 20

In [ ]:
# single task BayesOpt
stbo_results = []
for i in range(NUM_REPEATS):
    print(f'STBO repeat number : {i+1}')
    exp = MIT_case1(noise_level=1)
    result = run_stbo(exp, max_iterations=MAX_ITER)
    stbo_results.append(result)
    clear_output(wait=True)

In [ ]:
# prep the meta-data --> 

pt_data_2[('task', 'METADATA')] = np.zeros(pt_data_2.shape[0], dtype=int)

In [ ]:
pt_data_2

In [ ]:
# multi-task BayesOpt - one auxillary task (MIT case 2)
mtbo_results = []
for i in range(NUM_REPEATS):
    print(f'MTBO repeat number : {i+1}')
    exp = MIT_case1(noise_level=1)
    result = run_mtbo(exp, pt_data_2, max_iterations=MAX_ITER)
    mtbo_results.append(result)
    clear_output(wait=True)

In [ ]:
# multi-task BayesOpt with 8 points from all the tasks

# prepare all the data (#2 already prepared in the correct format)
pt_data_3[('task', 'METADATA')] = np.zeros(pt_data_3.shape[0], dtype=int)+1
pt_data_4[('task', 'METADATA')] = np.zeros(pt_data_4.shape[0], dtype=int)+2
pt_data_5[('task', 'METADATA')] = np.zeros(pt_data_5.shape[0], dtype=int)+3

pt_data_all_meta = pd.concat([pt_data_2, pt_data_3, pt_data_4, pt_data_5])
pt_data_all_meta.head()

In [ ]:
# multi-task BayesOpt - one auxillary task (MIT case 2)
mtbo_all_results = []
for i in range(NUM_REPEATS):
    print(f'MTBO repeat number : {i+1}')
    exp = MIT_case1(noise_level=1)
    result = run_mtbo(exp, pt_data_all_meta, max_iterations=MAX_ITER)
    mtbo_all_results.append(result)
    #clear_output(wait=True)
    


## Plot results

In [ ]:
mtbo_dfs = []
for run_ix, r in enumerate(mtbo_results):
    objective = r.experiment.domain.output_variables[0].name
    yields = r.experiment.data[objective].tolist()
    dict_ = {'iter': np.arange(21)+1, 'yield': yields}
    df = pd.DataFrame(dict_)
    df['run_ix'] = run_ix
    df['cummax_yield'] = df['yield'].cummax()
    df['regret'] = 1.0 - df['cummax_yield'] 
    mtbo_dfs.append(df)
    
mtbo_df = pd.concat(mtbo_dfs)

mtbo_all_dfs = []
for run_ix, r in enumerate(mtbo_all_results):
    objective = r.experiment.domain.output_variables[0].name
    yields = r.experiment.data[objective].tolist()
    dict_ = {'iter': np.arange(21)+1, 'yield': yields}
    df = pd.DataFrame(dict_)
    df['run_ix'] = run_ix
    df['cummax_yield'] = df['yield'].cummax()
    df['regret'] = 1.0 - df['cummax_yield'] 
    mtbo_all_dfs.append(df)
    
mtbo_all_df = pd.concat(mtbo_all_dfs)


In [ ]:
# comparison plots
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sns.lineplot(data=mtbo_df, x='iter', y='cummax_yield')
sns.lineplot(data=mtbo_all_df, x='iter', y='cummax_yield')

In [ ]:
# comparison plots
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sns.lineplot(data=mtbo_df, x='iter', y='regret')
sns.lineplot(data=mtbo_all_df, x='iter', y='regret')

ax.set_yscale('log')